<a href="https://colab.research.google.com/github/iguerrasevillano/TFM/blob/main/TLDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install xmltodict

In [ ]:
import xml.etree.ElementTree as ET
import xmltodict
import json

# Specify the path to your XML file
xml_file_path = path + 'W99-0629.xml'

# Parse the XML file
tree = ET.parse(xml_file_path)

# Get the root element
root = tree.getroot()

xml_string = ET.tostring(root, encoding="utf-8").decode("utf-8")

print(xml_string)

data_dict = xmltodict.parse(xml_string)

# Convert the dictionary to a JSON string
json_string = json.dumps(data_dict, indent=2)

# Print or use the resulting JSON string
print(json_string)

<PAPER>
  <S sid="0">Cascaded Grammatical Relation Assignment</S>
  <ABSTRACT>
    <S sid="1" ssid="1">In this paper we discuss cascaded Memory- Based grammatical relations assignment.</S>
    <S sid="2" ssid="2">In the first stages of the cascade, we find chunks of several types (NP,VP,ADJP,ADVP,PP) and label them with their adverbial function (e.g. local, temporal).</S>
    <S sid="3" ssid="3">In the last stage, we assign grammatical relations to pairs of chunks.</S>
    <S sid="4" ssid="4">We studied the effect of adding several levels to this cascaded classifier and we found that even the less performing chunkers enhanced the performance of the relation finder.</S>
  </ABSTRACT>
  <SECTION title="1 Introduction" number="1">
    <S sid="5" ssid="1">When dealing with large amounts of text, finding structure in sentences is often a useful preprocessing step.</S>
    <S sid="6" ssid="2">Traditionally, full parsing is used to find structure in sentences.</S>
    <S sid="7" ssid="3">Howe

In [ ]:
tree

In [ ]:
import os

os.getcwd()

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
BASE_PATH = "/content/drive/MyDrive/VIU/TFM/Desarrollo/Data/"

documents = os.listdir(BASE_PATH)

In [ ]:
os.listdir(BASE_PATH)

['top1000_complete', 'TLDR']

In [ ]:
path = BASE_PATH + 'W99-0629/'+ 'Documents_xml/'

In [ ]:
###########################33

In [120]:
import pandas as pd

jsonl_file_path = BASE_PATH + 'TLDR/train.jsonl'

# Read the JSON Lines file into a list of dictionaries
data_list = []
with open(jsonl_file_path, 'r') as jsonl_file:
    for line in jsonl_file:
        data_dict = json.loads(line)
        data_list.append(data_dict)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data_list)

# Display the DataFrame
display(df.head())
print(df['source'][0])

,source,source_labels,rouge_scores,paper_id,target
0,[Due to the success of deep learning to solvin...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.3018867874688502, 0.37209301838831804, 0.60...",SysEexbRb,[We provide necessary and sufficient analytica...
1,[The backpropagation (BP) algorithm is often t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.13043477920604923, 0.142857139229...",SygvZ209F7,"[Biologically plausible learning algorithms, p..."
2,"[We introduce the 2-simplicial Transformer, an...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.33333332839506175, 0.8888888839111112, 0.11...",rkecJ6VFvr,[We introduce the 2-simplicial Transformer and...
3,"[We present Tensor-Train RNN (TT-RNN), a novel...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.06666666222222252, 0.06451612466181092, 0.0...",HJJ0w--0W,[Accurate forecasting over very long time hori...
4,[Recent efforts on combining deep models with ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.2777777727932099, 0.5714285666581633, 0.095...",HyH9lbZAW,[We propose a variational message-passing algo...


['Due to the success of deep learning to solving a variety of challenging machine learning tasks, there is a rising interest in understanding loss functions for training neural networks from a theoretical aspect.', 'Particularly, the properties of critical points and the landscape around them are of importance to determine the convergence performance of optimization algorithms.', 'In this paper, we provide a necessary and sufficient characterization of the analytical forms for the critical points (as well as global minimizers) of the square loss functions for linear neural networks.', 'We show that the analytical forms of the critical points characterize the values of the corresponding loss functions as well as the necessary and sufficient conditions to achieve global minimum.', 'Furthermore, we exploit the analytical forms of the critical points to characterize the landscape properties for the loss functions of linear neural networks and shallow ReLU networks.', 'One particular conclu

In [121]:
df['source'] = df['source'].apply(lambda x : ' '.join(x))
df['target'] = df['target'].apply(lambda x : ' '.join(x))

no_need_columns = ['source_labels', 'rouge_scores']
df.drop(columns=no_need_columns)

,source,paper_id,target
0,Due to the success of deep learning to solving...,SysEexbRb,We provide necessary and sufficient analytical...
1,The backpropagation (BP) algorithm is often th...,SygvZ209F7,"Biologically plausible learning algorithms, pa..."
2,"We introduce the 2-simplicial Transformer, an ...",rkecJ6VFvr,We introduce the 2-simplicial Transformer and ...
3,"We present Tensor-Train RNN (TT-RNN), a novel ...",HJJ0w--0W,Accurate forecasting over very long time horiz...
4,Recent efforts on combining deep models with p...,HyH9lbZAW,We propose a variational message-passing algor...
...,...,...,...
1987,"Semi-supervised learning, i.e. jointly learnin...",rJel41BtDH,Pseudo-labeling has shown to be a weak alterna...
1988,Model-free reinforcement learning (RL) has bee...,Skw0n-W0Z,We show that a special goal-condition value fu...
1989,We introduce a neural architecture to perform ...,rJgFtkhEtr,A novel neural architecture for efficient amor...
1990,Machine learned large-scale retrieval systems ...,SJxPVcSonN,We propose a novel two-tower shared-bottom mod...


In [122]:
df['count_target'] = df['target'].apply(lambda x : len(x.split()))
df['count_source'] = df['source'].apply(lambda x : len(x.split()))

In [128]:
aux = df[df['count_target']>=15]
aux.shape

(1433, 7)

In [117]:
df['count_target'].mean()

51.78836833602585